In [82]:
import numpy as np
import os
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
import csv

Variables

In [ ]:
kernel_num = 100
kernel_vec = [3, 4, 5]
epochs = 10
lr = 0.01

* We are using Restaurant-large dataset.
* Here we are loading the train and test datasets.
* Also we are extracting sentences, aspects and sentiments.

In [83]:
# load the json file into a list
with open('./atsa-restaurant/atsa_train.json','rb') as f:
    data1 = json.load(f)

with open('./atsa-restaurant/atsa_test.json','rb') as f:
    data2 = json.load(f)

sentence_data = [x['sentence'] for x in data1] + [x['sentence'] for x in data2]
aspect_data = [x['aspect'] for x in data1] + [x['aspect'] for x in data2]
sentiment_data = [x['sentiment'] for x in data1] + [x['sentiment'] for x in data2]

sentence_data, aspect_data, sentiment_data = shuffle(sentence_data, aspect_data, sentiment_data)

# print('Number of sentences: ', len(sentence_data))
# print('Number of aspects: ', len(aspect_data))
# print('Number of sentiments: ', len(sentiment_data))

* In the following cell, we are just making a wordcount of each word in the sentences
* We are removing symbols at end of each word.
* If the word occurs for the first time, we assign the count to zero
* else we increase the count

In [84]:
data_words = {}
for example in sentence_data:
    for word in example.split():
        if word[-1] in ['.',',','!','?']:
            word = word[:-1]
        if word not in data_words:
            data_words[word] = 0
        else:
            data_words[word] += 1

* Similar to the above cell, we perform the same operations for aspect categories

In [85]:
aspect_categories = {}
for example in aspect_data:
    for word in example.split():
        if word[-1] in ['.',',','!','?']:
            word = word[:-1]
        if word not in aspect_categories:
            aspect_categories[word] = 0
        else:
            aspect_categories[word] += 1

In [86]:
print(len(data_words))
print(len(aspect_categories))

4818
1387


* In the following cell we are loading the glove file and encoding vectors for words present in our data.
* `glove_word_to_vec_map:` All words in glove file and the corresponding encodings, 
* `data_word_to_vec_map:`All words in our data and the corresponding encodings


In [87]:
# Load glove vectors
glove_folder = os.path.join(os.getcwd(), 'glove_file')

# get path of glove.6B.300d.txt file in test folder
glove_file = os.path.join(glove_folder, 'glove.6B.300d.txt')

def load_glove_vectors(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    with open(glove_file, 'r', encoding="utf8") as f:
        embs = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]

            if curr_word in data_words:
                try:
                    embedding = np.array([float(value) for value in line[1:]])
                    embs[curr_word] = embedding
                except:
                    print('error loading embedding')
    return words, word_to_vec_map, embs

glove_words, glove_word_to_vec_map, data_word_to_vec_map = load_glove_vectors(glove_file)
# print(len(words))
# print(len(word_to_vec_map))
# print(word_to_vec_map['the'])
# print(word_to_vec_map['the'].shape)


In [88]:
def get_aspect_catogories_glove_embedding(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        embs = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]

            if curr_word in aspect_categories:
                try:
                    embedding = np.array([float(value) for value in line[1:]])
                    embs[curr_word] = embedding
                except:
                    print('error loading embedding')
    return embs

aspect_catogories_to_vec_map = get_aspect_catogories_glove_embedding(glove_file)

In [89]:
unknown_word_vector = np.mean(list(glove_word_to_vec_map.values()), axis=0)

In [90]:
print(len(data_words))
print(len(data_word_to_vec_map))
missing_words = len(data_words) - len(data_word_to_vec_map)
print(missing_words)

4818
4044
774


In [91]:
print(len(aspect_categories))
print(len(aspect_catogories_to_vec_map))
missing_aspect_categories_words = len(aspect_catogories_to_vec_map) - len(aspect_categories)
print(missing_aspect_categories_words)

1387
1253
-134


In [92]:
embedding_matrix = []
idx2word = []
word2idx = {}
embedding_matrix.append(np.zeros(300)) # this will be our zero padding for the network
idx2word.append('')
word2idx[''] = 0
for i, (word, emb) in enumerate(data_word_to_vec_map.items()):
    embedding_matrix.append(emb)
    idx2word.append(word)
    word2idx[word] = i + 1
    # word2idx[word] = i
embedding_matrix = np.asarray(embedding_matrix)

In [93]:
ac_embedding_matrix = []
ac_idx2word = []
ac_word2idx = {}
# ac_embedding_matrix.append(np.zeros(300)) # this will be our zero padding for the network
# ac_idx2word.append('')
# ac_word2idx[''] = 0
for i, (word, emb) in enumerate(aspect_catogories_to_vec_map.items()):
    ac_embedding_matrix.append(emb)
    ac_idx2word.append(word)
    # ac_word2idx[word] = i + 1
    ac_word2idx[word] = i
ac_embedding_matrix = np.asarray(ac_embedding_matrix)

In [94]:
x_train = []
for example in sentence_data:
    temp = []
    for word in example.split():
        if word[-1] in ['.',',','!','?']:
            word = word[:-1]
        if word in word2idx:
            temp.append(word2idx[word])
    # if len(temp) == 0:
    #     print(example)
    x_train.append(temp)

In [95]:
ac_train = []
for example in aspect_data:
    temp = []
    for word in example.split():
        if word[-1] in ['.',',','!','?']:
            word = word[:-1]
        if word in ac_word2idx:
            temp.append(ac_word2idx[word])
    # if len(temp) == 0:
    #     print(example)
    ac_train.append(temp)

In [96]:
x_train = np.asarray(x_train, dtype=object)

In [97]:
ac_train = np.asarray(ac_train, dtype=object)

In [98]:
print(x_train.shape)

(4827,)


In [99]:
print(ac_train.shape)

(4827,)


In [100]:
max_length = 0
min_length = 1000
for example in x_train:
    if len(example) > max_length:
        max_length = len(example)
    if len(example) < min_length:
        min_length = len(example)

In [101]:
print(max_length)
print(min_length)

64
1


In [102]:
a_max_length = 0
a_min_length = 1000
for example in ac_train:
    if len(example) > a_max_length:
        a_max_length = len(example)
    if len(example) < a_min_length:
        a_min_length = len(example)

In [103]:
print(a_max_length)
print(a_min_length)

16
0


In [104]:
total_length = 0
for i in range(len(x_train)):
    total_length += len(x_train[i])
avg_length = total_length / len(x_train)
print(avg_length)

16.64284234514191


In [105]:
a_total_length = 0
for i in range(len(ac_train)):
    a_total_length += len(ac_train[i])
a_avg_length = a_total_length / len(ac_train)
print(a_avg_length)

1.3706235757199088


In [106]:
for i in range(len(x_train)):
    x_train[i] = np.pad(x_train[i], (max_length - len(x_train[i]), 0), 'constant')

In [107]:
for i in range(len(ac_train)):
    ac_train[i] = np.pad(ac_train[i], (a_max_length - len(ac_train[i]), 0), 'constant')

In [108]:
x_train_data = []
for x in x_train:
    x_train_data.append([k for k in x])

x_train_data = np.array(x_train_data)


In [109]:
ac_train_data = []
for x in ac_train:
    ac_train_data.append([k for k in x])

ac_train_data = np.array(ac_train_data)


In [110]:
print(embedding_matrix.shape)
print(len(data_word_to_vec_map))
print(len(data_words))
print(np.array(sentence_data).shape)
print(np.array(aspect_data).shape)
print(ac_embedding_matrix.shape)

(4045, 300)
4044
4818
(4827,)
(4827,)
(1253, 300)


In [111]:
print(x_train_data.shape)
print(ac_train_data.shape)

(4827, 64)
(4827, 16)


In [112]:
print(embedding_matrix.shape)

(4045, 300)


In [113]:
class CNN_Gate_Aspect_Text(nn.Module):
    def __init__(self, embedding_matrix, class_num, kernel_num, kernel_sizes, aspect_matrix):
        super(CNN_Gate_Aspect_Text, self).__init__()
        
        V = embedding_matrix.shape[0]
        D = embedding_matrix.shape[1]
        C = class_num
        A = aspect_matrix.shape[0]

        Co = kernel_num
        Ks = kernel_sizes

        self.embed = nn.Embedding(V, D)
        self.embed.load_state_dict({'weight': torch.tensor(embedding_matrix)})
        # self.embed.weight = nn.Parameter(embedding_matrix, requires_grad=True)
        self.embed.weight.requires_grad = True

        self.aspect_embed = nn.Embedding(A, aspect_matrix.shape[1])
        self.aspect_embed.load_state_dict({'weight':  torch.tensor(aspect_matrix)})
        # self.aspect_embed.weight = nn.Parameter(aspect_matrix, requires_grad=True)
        self.aspect_embed.weight.requires_grad = True

        self.convs1 = nn.ModuleList([nn.Conv1d(D, Co, K) for K in Ks])
        self.convs2 = nn.ModuleList([nn.Conv1d(D, Co, K) for K in Ks])
        self.convs3 = nn.ModuleList([nn.Conv1d(D, Co, K, padding=K-2) for K in [3]])


        # self.convs3 = nn.Conv1d(D, 300, 3, padding=1), smaller is better
        self.dropout = nn.Dropout(0.2)

        self.fc1 = nn.Linear(len(Ks)*Co, C)
        self.fc_aspect = nn.Linear(100, Co)


    def forward(self, feature, aspect):
        feature = self.embed(feature)  # (N, L, D)
        aspect_v = self.aspect_embed(aspect)  # (N, L', D)
        aa = [F.relu(conv(aspect_v.transpose(1, 2))) for conv in self.convs3]  # [(N,Co,L), ...]*len(Ks)
        aa = [F.max_pool1d(a, a.size(2)).squeeze(2) for a in aa]
        aspect_v = torch.cat(aa, 1)
        # aa = F.tanhshrink(self.convs3(aspect_v.transpose(1, 2)))  # [(N,Co,L), ...]*len(Ks)
        # aa = F.max_pool1d(aa, aa.size(2)).squeeze(2)
        # aspect_v = aa
        # smaller is better

        x = [F.tanh(conv(feature.transpose(1, 2))) for conv in self.convs1]  # [(N,Co,L), ...]*len(Ks)
        y = [F.relu(conv(feature.transpose(1, 2)) + self.fc_aspect(aspect_v).unsqueeze(2)) for conv in self.convs2]
        x = [i*j for i, j in zip(x, y)]

        # pooling method
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N,Co), ...]*len(Ks)
        # x = [F.adaptive_max_pool1d(i, 2) for i in x]
        # x = [i.view(i.size(0), -1) for i in x]

        x = torch.cat(x, 1)
        x = self.dropout(x)  # (N,len(Ks)*Co)
        logit = self.fc1(x)  # (N,C)
        return logit, x, y

In [114]:
sentiments = {}

# get unique sentiments in sentiment data
for sentiment in sentiment_data:
    if sentiment not in sentiments:
        sentiments[sentiment] = 1
    else:
        sentiments[sentiment] += 1

print(sentiments)

sentiment_input = []
for sentiment in sentiment_data:
    if sentiment == 'positive':
        sentiment_input.append(2)
    elif sentiment == 'negative':
        sentiment_input.append(0)
    else:
        sentiment_input.append(1)

sentiment_input = np.array(sentiment_input)
print(sentiment_input.shape)

{'neutral': 829, 'negative': 1001, 'positive': 2892, 'conflict': 105}
(4827,)


In [115]:
# dataset = CustomImageDataset(x_train_data, labels)

train_length = int(len(sentence_data) * 0.8) # 80% training data, 20% test data
test_length = len(sentence_data) - train_length

# print(x_train_data.shape)

# concatenate the x_train_data, ac_train_data and sentiment_input
x_train_data = np.concatenate((x_train_data, ac_train_data), axis=1)
# print(x_train_data.shape)
x_train_data = np.concatenate((x_train_data, sentiment_input.reshape(-1, 1)), axis=1)

# print(x_train_data.shape)



# print(len(x_train_dataloader) * batch_size)
# print(len(y_test_dataloader) * batch_size)

In [116]:
# split x_train_data into training and test data using train_test_split
x_train, x_test = train_test_split(x_train_data, test_size=0.2, random_state=42)

print(x_train.shape)

(3861, 81)


In [117]:
batch_size = 32
# x_train = torch.tensor(x_train.astype('float64')).to(torch.int64)
train_batches = DataLoader(torch.Tensor(x_train).to(dtype=torch.long), batch_size=batch_size, shuffle=True)

# for data in train_batches:
#     # print(data)
#     # break
# # convert data to int tensor
#     # data = data.to(torch.int64)
#     print(data.shape)
#     print(data[:, :-2].shape)
#     break

In [118]:
test_batches = DataLoader(torch.Tensor(x_test).to(dtype=torch.long), batch_size=batch_size, shuffle=True)

In [119]:
def tokens_to_string(tokens):  # Convert tokens back into their sting value
    words = [idx2word[token] for token in tokens]
    text = " ".join(words)
    return text

In [ ]:
def ac_tokens_to_string(tokens):  # Convert tokens back into their sting value
    words = [ac_idx2word[token] for token in tokens]
    text = " ".join(words)
    return text

In [120]:
def train():

    loss_function = nn.CrossEntropyLoss()
    model = CNN_Gate_Aspect_Text(torch.Tensor(embedding_matrix).to(dtype=torch.long), len(sentiments), kernel_num, kernel_vec, torch.Tensor(ac_embedding_matrix).to(dtype=torch.long))
    # model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    

    for epoch in range(epochs):
        for i, data in enumerate(train_batches):
            sentence = data[:, :-(a_max_length+1)]
            aspect = data[:, -(a_max_length+1):-1]
            sentiment = data[:, -1]

            # print(sentence.shape)
            # print(aspect.shape)
            # print(sentiment.shape)

            # for i in range(len(sentence)):
            #     print(tokens_to_string(sentence[i].to(dtype=torch.long, device='cpu')))
            #     print(ac_idx2word[aspect[i].to(dtype=torch.long, device='cpu')])
            #     print(sentiment[i])

            # print(aspect[0])

            # x = x.to(device)
            # y = y.to(device)
            # optimizer.zero_grad()
            model.zero_grad()
            # model.zero_grad()
            # convert sentence to int tensor
            sentence = sentence.to(dtype=torch.long)
            aspect = aspect.to(dtype=torch.long)
            
            logit, x, y = model(sentence, aspect)
            # y = torch.tensor(y)
            loss = loss_function(logit, sentiment)
            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                print('Epoch: ', epoch, 'Batch: ', i, 'Loss: ', loss.item())
            
    return model

model = train()

C:\Users\srisa\AppData\Local\Temp\ipykernel_7528\3799450476.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embed.load_state_dict({'weight': torch.tensor(embedding_matrix)})
C:\Users\srisa\AppData\Local\Temp\ipykernel_7528\3799450476.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.aspect_embed.load_state_dict({'weight':  torch.tensor(aspect_matrix)})


Epoch:  0 Batch:  0 Loss:  1.0995526313781738
Epoch:  0 Batch:  100 Loss:  0.7206839323043823
Epoch:  1 Batch:  0 Loss:  0.7448155879974365
Epoch:  1 Batch:  100 Loss:  0.31195175647735596
Epoch:  2 Batch:  0 Loss:  0.3568362295627594
Epoch:  2 Batch:  100 Loss:  0.420025497674942
Epoch:  3 Batch:  0 Loss:  0.23110097646713257
Epoch:  3 Batch:  100 Loss:  0.32050710916519165
Epoch:  4 Batch:  0 Loss:  0.220163956284523
Epoch:  4 Batch:  100 Loss:  0.20904314517974854
Epoch:  5 Batch:  0 Loss:  0.15926630795001984
Epoch:  5 Batch:  100 Loss:  0.16802304983139038
Epoch:  6 Batch:  0 Loss:  0.021402699872851372
Epoch:  6 Batch:  100 Loss:  0.20569710433483124
Epoch:  7 Batch:  0 Loss:  0.11067144572734833
Epoch:  7 Batch:  100 Loss:  0.14777164161205292
Epoch:  8 Batch:  0 Loss:  0.04526834562420845
Epoch:  8 Batch:  100 Loss:  0.041393280029296875
Epoch:  9 Batch:  0 Loss:  0.040724147111177444
Epoch:  9 Batch:  100 Loss:  0.01950805075466633


In [121]:
# evaluate the model

def print_accuracy(dataloader):
    correct = 0
    total = 0
    visual =[]
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            sentence = data[:, :-(a_max_length+1)]
            aspect = data[:, -(a_max_length+1):-1]
            sentiment = data[:, -1]
            outputs, _, _ = model(sentence, aspect)
            outputs = outputs.detach().cpu().numpy()
            sentiment = sentiment.detach().cpu().numpy()
            # output the labels with highest probability
            outputs = np.argmax(outputs, axis=1)
            # print(outputs)
            # labels = labels == 1.0
            # use outputs to calculate accuracy
            correct += np.sum(outputs == sentiment)
            total += len(outputs)
            for i in range(len(sentence)):
                visual.append([tokens_to_string(sentence[i].tolist()), ac_tokens_to_string(aspect[i].tolist()), sentiment[i], outputs[i]])
    with open('visual.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(visual)
    return correct / total

In [122]:
print(print_accuracy(train_batches))

0.9849779849779849


In [123]:
print(print_accuracy(test_batches))

0.727743271221532
